In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [2]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated, List
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage

In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="openai/gpt-oss-20b",
    temperature=0
)

In [20]:
from langchain_google_genai import ChatGoogleGenerativeAI
google_lm = ChatGoogleGenerativeAI(
    model="models/gemini-2.5-flash",
    temperature=0.7
)

In [23]:
PLAN_PROMPT = """
    You are the CMO of a reputed bank. \
        Your role is to guide a digital sales team whose job is to deliver personalized sales pitches to customers. \
        Provide clear, concise guidelines for the team, covering: tone of conversation, approach, key points to keep in mind, and best practices. \
        Present the response in short, actionable bullet points under 100 words.
"""

In [24]:
response = google_lm.invoke(PLAN_PROMPT)

print(response.content)

Here are your guidelines:

*   **Tone:** Be professional, empathetic, and confident. Act as a trusted financial advisor, not just a salesperson.
*   **Approach:** Actively listen to customer needs. Leverage data to personalize solutions, focusing on tangible value, not just features.
*   **Key Points:** Build trust by highlighting security and reliability. Clearly articulate how our offerings solve specific customer pain points.
*   **Best Practices:** Ensure timely and relevant follow-ups. Continuously learn and adapt your pitch. Maintain deep knowledge of all bank products.


In [25]:
response = llm.invoke(PLAN_PROMPT)

print(response.content)

- Use a friendly, confident tone; speak like a trusted advisor, not a hard‑seller.  
- Start with a quick, personalized greeting that references recent activity or goals.  
- Ask open‑ended questions to uncover needs, then listen actively.  
- Highlight benefits that align with the customer’s goals, not just features.  
- Keep language simple; avoid jargon unless the client asks for detail.  
- Offer a clear next step (demo, quote, or follow‑up call) with a deadline.  
- Record key insights in the CRM immediately; personalize future touchpoints.  
- Follow up promptly, thank them for their time, and confirm the next action.


In [26]:
previous_strategy = """- Use a friendly, confident tone; speak like a trusted advisor, not a hard seller.  
- Start with a quick, personalized greeting that references recent activity or goals.  
- Ask open‑ended questions to uncover needs, then listen actively.  
- Highlight benefits that align with the customer's goals, not just features.  
- Keep language simple; avoid jargon unless the client asks for detail.  
- Offer a clear next step (demo, quote, or follow‑up call) with a deadline.  
- Record key insights in the CRM immediately; personalize future touchpoints.  
- Follow up promptly, thank them for their time, and confirm the next action."""

suggestions = """Start the conversation with a hook and include story based conversation wherever possible"""


In [ ]:
from langchain.prompts import PromptTemplate

REFLECT_PROMPT = """
As the CMO of a reputed bank \
you have previously provided guidelines to the digital sales team on delivering personalized sales pitches to customers covering tone, approach, and key points. \
Now, the higher authorities have suggested changes to the marketing campaigns. \
Based on these suggestions, redefine and update the sales guidelines for the team. \
Include actionable, concise recommendations in bullet points, ensuring they align with the updated marketing strategy while maintaining personalization for customers. \
Present the response in short, actionable bullet points under 100 words.

-- Suggestions --
{suggestions}

-- Previous Strategy --
{previous_strategy}
"""

reflect_template = PromptTemplate(
    input_variables=["suggestions", "previous_strategy"],
    template=REFLECT_PROMPT
)

In [42]:
reflect_prompt = reflect_template.format(
    suggestions=suggestions,
    previous_strategy=previous_strategy
)

In [ ]:
print(reflect_prompt)


As the CMO of a reputed bank you have previously provided guidelines to the digital sales team on delivering personalized sales pitches to customers covering tone, approach, and key points. Now, the higher authorities have suggested changes to the marketing campaigns. Based on these suggestions, redefine and update the sales guidelines for the team. Include actionable, concise recommendations in bullet points, ensuring they align with the updated marketing strategy while maintaining personalization for customers. Present the response in short, actionable bullet points under 100 words.

-- Suggestions --
Start the conversation with a hook and include story based conversation wherever possible

-- Previous Strategy --
- Use a friendly, confident tone; speak like a trusted advisor, not a hard seller.  
- Start with a quick, personalized greeting that references recent activity or goals.  
- Ask open‑ended questions to uncover needs, then listen actively.  
- Highlight benefits that align

In [ ]:
response = llm.invoke(reflect_prompt)
print(response.content)

- Begin with a compelling hook: a quick anecdote or surprising fact that ties to the client’s industry.  
- Use a story‑driven opener that mirrors the client’s recent milestone or challenge.  
- Keep the tone friendly, confident, and advisory—no hard selling.  
- Personal greeting that mentions a recent activity or goal.  
- Ask open‑ended questions; listen actively and note insights in CRM instantly.  
- Highlight benefits aligned with the client’s objectives, using simple language.  
- Offer a clear next step (demo, quote, call) with a specific deadline.  
- Follow up promptly, thank them, and confirm the next action.


In [46]:
response = google_lm.invoke(reflect_prompt)
print(response.content)

Here are the updated sales guidelines:

*   **Engage with a Hook & Story:** Start conversations with a captivating hook, then weave in concise, relatable customer success stories to illustrate value.
*   **Be a Trusted Advisor:** Maintain a friendly, confident tone, focusing on problem-solving, not hard selling.
*   **Personalize & Uncover Needs:** Reference customer activity, then ask open questions to deeply understand their unique goals.
*   **Narrate Benefits:** Frame benefits through engaging narratives that directly align with the customer's aspirations.
*   **Simplify & Guide:** Use clear, jargon-free language. Offer a simple, specific next step.
*   **CRM for Continuity:** Immediately record insights to ensure personalized future touchpoints and prompt follow-ups.


In [71]:
guidelines = """- Begin with a compelling hook: a quick anecdote or surprising fact that ties to the client's industry.  
- Use a story‑driven opener that mirrors the client's recent milestone or challenge.  
- Keep the tone friendly, confident, and advisory—no hard selling.  
- Personal greeting that mentions a recent activity or goal.  
- Ask open‑ended questions; listen actively and note insights in CRM instantly.  
- Highlight benefits aligned with the client's objectives, using simple language.  
- Offer a clear next step (demo, quote, call) with a specific deadline.  
- Follow up promptly, thank them, and confirm the next action."""

In [69]:
customer_info = """
            age - 34,
            gender - Female,
            marital_status - Married,
            children - 2,
            occupation - IT Project Manager,
            annual_income - 450000
            loan_status - 
                active yes,
                type Home Loan,
                balance 4500000
            credit_card -
                has_card: yes,
                type: Premium
            savings_balance: 320000,
            investment_products: Mutual Funds, Fixed Deposit
            """

product_to_market = """SIPs, Personal Loan"""

NOTIFICATION_WRITER = """You are a content writer for a reputed bank \
        specializing in creating engaging and persuasive content that sparks customer curiosity. \
        Your task is to write short Telegram messages or notifications that grab the customer's attention and act as a starting point to introduce them to a product. \
        Keep the messages concise, friendly, and curiosity-driven \
        provide output in a json format:
        
        -- Market this product to them 
        {product_to_market}
        
        -- Here is the brieft about customer
        {customer_info}
        
        -- Guideline
        {guidelines}
        """


In [70]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
output_parser=JsonOutputParser()

class Notification(BaseModel):
    product1: str = Field(description="Notification Focused on SIPs")
    product2: str = Field(description="Notification Focused on Personal Loan")

json_parser = JsonOutputParser(pydantic_object=Notification)


In [72]:
noifi_writer_template = PromptTemplate(
    input_variables=["product_to_market", "customer_info", "guidelines"],
    template=NOTIFICATION_WRITER,
    partial_variables={"format_instructions": json_parser.get_format_instructions()},
)

In [73]:
notifi_writer_prompt = noifi_writer_template.format(
    product_to_market = product_to_market, 
    customer_info = customer_info,
    guidelines=guidelines
)

In [74]:
chain = noifi_writer_template | llm | json_parser

In [76]:
response = chain.invoke({"product_to_market": product_to_market, "customer_info": customer_info, "guidelines": guidelines})

print(response)

{'messages': [{'product': 'SIP', 'message': "Hi Priya, did you know that a ₹500/month SIP can grow into a substantial nest egg by the time your kids finish college? 🎯 You just celebrated your second child's birthday—wondering how to secure their future? Let’s chat for 10 minutes before Friday to explore a personalized SIP plan. I’ll note your preferences in our system and send a calendar invite. Looking forward!"}, {'product': 'Personal Loan', 'message': 'Hey Priya, ever thought a flexible loan could make that dream home renovation or family vacation a reality? 🏡✈️ What’s the biggest project you’re eyeing this year? Let’s connect for a quick call by Thursday to discuss low‑interest, quick‑approval options. I’ll capture your insights in our CRM and send you a confirmation. Thanks!'}]}


In [101]:
Email_WRITER = """You are a content writer for a reputed bank, specializing in creating engaging and persuasive content. \
    Your task is to write email campaigns targeted at young customers. \
    The emails should be concise, relatable, and designed to capture attention quickly. \
    Focus on: subject lines that spark curiosity, friendly yet professional tone, clear value for the customer, and encouraging them to take action \
    just provide the body of an email no greetings
    provide one email only as a json format
    subject, body
    
    -- user info
    {customer_info}

    -- product to market\
    {product_to_market}
    
    -- Guideline
    {guidelines}
    """

email_writer_template = PromptTemplate(
    input_variables=["product_to_market", "customer_info", "guidelines"],
    template=Email_WRITER,
    partial_variables={"format_instructions": json_parser.get_format_instructions()},
)

In [102]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
output_parser=JsonOutputParser()

class Email(BaseModel):
    subject: str = Field(description="Subject of an email")
    body: str = Field(description="Body of an email")

email_parser = JsonOutputParser(pydantic_object=Email)

email_chain = email_writer_template | llm | email_parser

In [103]:
email_writer_prompt = email_writer_template.format(
    product_to_market = product_to_market, 
    customer_info = customer_info,
    guidelines=guidelines
)

In [104]:
response = email_chain.invoke({"product_to_market": product_to_market, "customer_info": customer_info, "guidelines": guidelines})

print(response)

{'subject': 'Turn your project‑level budgeting into family‑level growth', 'body': "Did you know that the average IT project manager spends 30% of their time juggling budgets, yet only 10% of that time is actually invested in their family's future?\n\nLast month you just wrapped up a major milestone—congratulations! Imagine that success translating into a smoother home‑loan repayment or a new car for the kids.\n\nWhat would it feel like to have a steady stream of returns without the hassle of market timing?\n\n**SIPs** give you disciplined, low‑entry investing that grows with your savings, while **personal loans** offer flexible terms and lower rates to consolidate debt or fund a big purchase.\n\nLet’s schedule a 15‑minute chat by Friday, 5\u202fpm, to explore a tailored SIP plan and a personal‑loan option that fits your cash flow. Just reply “Ready” and we’ll lock in the slot.\n\nThanks for trusting us to help you build a brighter future for your family."}


In [105]:
print("Subject: ", response['subject'])
print("Body: ", response['body'])

Subject:  Turn your project‑level budgeting into family‑level growth
Body:  Did you know that the average IT project manager spends 30% of their time juggling budgets, yet only 10% of that time is actually invested in their family's future?

Last month you just wrapped up a major milestone—congratulations! Imagine that success translating into a smoother home‑loan repayment or a new car for the kids.

What would it feel like to have a steady stream of returns without the hassle of market timing?

**SIPs** give you disciplined, low‑entry investing that grows with your savings, while **personal loans** offer flexible terms and lower rates to consolidate debt or fund a big purchase.

Let’s schedule a 15‑minute chat by Friday, 5 pm, to explore a tailored SIP plan and a personal‑loan option that fits your cash flow. Just reply “Ready” and we’ll lock in the slot.

Thanks for trusting us to help you build a brighter future for your family.
